In [7]:
from pathlib import Path
import json
import pandas as pd

# Configure pandas display so long text is fully visible in tables
# pd.set_option('display.max_colwidth', None)
# pd.set_option('display.width', 0)

LOG_PATH = Path("/home/moon/Desktop/Financial_Document_Analyzer/data/logs/financebench_eval_20251125_094704.jsonl")
assert LOG_PATH.exists(), f"Log file not found: {LOG_PATH}"

rows = []
with LOG_PATH.open("r", encoding="utf-8") as fh:
    for line in fh:
        line = line.strip()
        if not line:
            continue
        rows.append(json.loads(line))
df = pd.DataFrame(rows)
df

,doc_name,question,ground_truth,question_type,question_reasoning,answer,citations,hits,eval_is_same,eval_reasoning
0,AMERICANEXPRESS_2022_10K,Which debt securities are registered to trade ...,There are none,domain-relevant,Information extraction,The provided text appears to be a regulatory f...,[],"[{'chunk_id': '943', 'source_doc': 'AMERICANEX...",False,None
1,AMERICANEXPRESS_2022_10K,What are the geographies that American Express...,"United States, EMEA, APAC, and LACC",domain-relevant,Information extraction,103 countries and territories as of 2022 (see ...,"[{'i': 6, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '1197', 'source_doc': 'AMERICANE...",False,None
2,AMERICANEXPRESS_2022_10K,Does AMEX have an improving operating margin p...,Performance is not measured through operating ...,domain-relevant,Numerical reasoning OR information extraction,American Express Holdings Limited,[],"[{'chunk_id': '19', 'source_doc': 'AMERICANEXP...",False,None
3,AMERICANEXPRESS_2022_10K,What drove gross margin change as of the FY202...,Performance is not measured through gross margin,domain-relevant,Logical reasoning (based on numerical reasonin...,The change in gross margin as of FY2022 for Am...,"[{'i': 6, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '19', 'source_doc': 'AMERICANEXP...",False,Answer 1 states that performance is not measur...
4,AMERICANEXPRESS_2022_10K,How much has the effective tax rate of America...,The effective tax rate for American Express ha...,domain-relevant,Numerical reasoning,The effective tax rate decreased by 3% from 24...,"[{'i': 2, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '354', 'source_doc': 'AMERICANEX...",True,None
5,AMERICANEXPRESS_2022_10K,What was the largest liability in American Exp...,Customer deposits,novel-generated,None,No answer,[],"[{'chunk_id': '847', 'source_doc': 'AMERICANEX...",False,None
6,AMERICANEXPRESS_2022_10K,Was American Express able to retain card membe...,Yes,novel-generated,None,"Yes, according to American Express's Annual Re...","[{'i': 3, 'source_doc': 'AMERICANEXPRESS_2022_...","[{'chunk_id': '35', 'source_doc': 'AMERICANEXP...",True,None


In [2]:
# Basic metrics
total = len(df)
correct = int(df['eval_is_same'].sum())
accuracy = correct / total if total else 0
print(f'Total questions: {total}')
print(f'Correct: {correct} ({accuracy:.2%})')

print('\nAccuracy by question_type:')
grouped = df.groupby('question_type')['eval_is_same'].agg(['mean', 'count'])
display(grouped.rename(columns={'mean': 'accuracy'}))


Total questions: 7
Correct: 2 (28.57%)

Accuracy by question_type:


,accuracy,count
question_type,,
domain-relevant,0.2,5
novel-generated,0.5,2


In [12]:
# Inspect incorrect predictions
bad_cases = df[df['eval_is_same'] == False]
print(f'Bad cases: {len(bad_cases)}')
bad_cases[['question', 'question_type', 'answer', 'ground_truth']]


Bad cases: 5


,question,question_type,answer,ground_truth
0,Which debt securities are registered to trade ...,domain-relevant,The provided text appears to be a regulatory f...,There are none
1,What are the geographies that American Express...,domain-relevant,103 countries and territories as of 2022 (see ...,"United States, EMEA, APAC, and LACC"
2,Does AMEX have an improving operating margin p...,domain-relevant,American Express Holdings Limited,Performance is not measured through operating ...
3,What drove gross margin change as of the FY202...,domain-relevant,The change in gross margin as of FY2022 for Am...,Performance is not measured through gross margin
5,What was the largest liability in American Exp...,novel-generated,No answer,Customer deposits


In [4]:
# Inspect a specific bad case (default: first failure)
idx = bad_cases.index[0] if len(bad_cases) else None
if idx is not None:
    rec = bad_cases.loc[idx]
    print('Question:', rec['question'])
    print('\nGround Truth:', rec['ground_truth'])
    print('\nModel Answer:', rec['answer'])
    print('\nCitations:', rec['citations'])
    print('\nHits (top-k chunk excerpts):')
    for hit in rec['hits']:
        print('-', hit.get('doc_id'), '|', (hit.get('text') or ''))
else:
    print('No bad cases available.')


Question: Which debt securities are registered to trade on a national securities exchange under American Express' name as of 2022?

Ground Truth: There are none

Model Answer: The provided text appears to be a regulatory filing by American Express Company, likely for the Securities and Exchange Commission (SEC). The content includes financial statements, regulatory ratios, capital adequacy calculations, and details of various plans and agreements related to employee compensation and benefits. The document seems to cover several years, including 2022 and 2021, providing insight into the company's financial performance and compliance with regulatory requirements.

Citations: []

Hits (top-k chunk excerpts):
- AMERICANEXPRESS_2022_10K_p176 | AMERICAN EXPRESS COMPANY AMERICAN EXPRESS NATIONAL BANK
- AMERICANEXPRESS_2022_10K_p1 | f ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934 For the fiscal year ended December 31, 2022 OR O TRANSITION REPORT PURSUANT 

In [11]:
# Inspect correct predictions
good_cases = df[df['eval_is_same'] == True]
print(f'Good cases: {len(good_cases)}')
good_cases[['question', 'question_type', 'answer', 'ground_truth']]

Good cases: 2


,question,question_type,answer,ground_truth
4,How much has the effective tax rate of America...,domain-relevant,The effective tax rate decreased by 3% from 24...,The effective tax rate for American Express ha...
6,Was American Express able to retain card membe...,novel-generated,"Yes, according to American Express's Annual Re...",Yes


In [6]:
# Inspect a specific good case (default: first failure)
idx = good_cases.index[0] if len(good_cases) else None
if idx is not None:
    rec = good_cases.loc[idx]
    print('Question:', rec['question'])
    print('\nGround Truth:', rec['ground_truth'])
    print('\nModel Answer:', rec['answer'])
    print('\nCitations:', rec['citations'])
    print('\nHits (top-k chunk excerpts):')
    for hit in rec['hits']:
        print("'chunk_id:'", hit.get('chunk_id'), '|', (hit.get('text') or ''))
else:
    print('No bad cases available.')


Question: How much has the effective tax rate of American Express changed between FY2021 and FY2022?

Ground Truth: The effective tax rate for American Express has changed/dropped from 24.6% in FY 2021 to 21.6% in FY 2022.

Model Answer: The effective tax rate decreased by 3% from 24.6% in FY2021 to 21.6% in FY2022.

Citations: [{'i': 2, 'source_doc': 'AMERICANEXPRESS_2022_10K', 'chunk_id': '275', 'element': 'text', 'page_start': 49, 'page_end': 49, 'text': 'The effective tax rate was 21.6 percent and 24.6 percent for 2022 and 2021, respectively. The reduction in the effective tax rate primarily reflected discrete tax benefits in the current year related to the resolution of prior year tax items. The tax rates in both years reflected the level of pretax income in relation to recurring permanent tax benefits and the geographic mix of business. 46'}]

Hits (top-k chunk excerpts):
'chunk_id:' 354 | Effective Minimum (a) Risk Based Capital Common Equity Tier 1 7.0 % American Express Compan